In [ ]:
import pandas as pd
import numpy as np

In [ ]:
supply_chain = pd.read_csv("supply_chain_updated.csv")
transactions = pd.read_csv("transactions.csv")

In [ ]:
supply_chain.shape , transactions.shape

In [ ]:
supply_chain.head()

In [ ]:
transactions.head()

In [ ]:
supply_chain.columns , transactions.columns

In [ ]:
supply_chain.dtypes

In [ ]:
transactions.dtypes

In [ ]:
transactions.info()

In [ ]:
supply_chain.info()

In [ ]:
np.sum(supply_chain.isnull().sum()), np.sum(transactions.isnull().sum())

In [ ]:
supply_chain.duplicated().sum(), transactions.duplicated().sum()

In [ ]:
supply_chain.describe()

In [ ]:
transactions.describe()

In [ ]:
# Create DIM_PRODUIT
dim_produit = supply_chain[['SKU', 'Category', 'ProductName', 'UnitPurchasePrice']].drop_duplicates()

# Rename columns to match schema
dim_produit = dim_produit.rename(columns={
    'UnitPurchasePrice': 'Price'
})

# Add Availability (StockLevel)
stock_levels = supply_chain.groupby('SKU')['StockLevel'].sum().reset_index()
dim_produit = pd.merge(dim_produit, stock_levels, on='SKU', how='left')

# Rename for consistency
dim_produit = dim_produit.rename(columns={
    'StockLevel': 'Availability'
})

dim_produit.head()

In [ ]:
dim_supplier = supply_chain[['SupplierName', 'SupplierCity', 'SupplierCountry']].drop_duplicates()

dim_supplier['SupplierID'] = [f"SUP-{i+1:05d}" for i in range(len(dim_supplier))]

dim_supplier = dim_supplier[['SupplierID', 'SupplierName', 'SupplierCity', 'SupplierCountry']]

dim_supplier.head()

In [ ]:
dim_transport_mode = supply_chain[['DeliveryCompany']].drop_duplicates()

dim_transport_mode['TransportModeID'] = [f"TRANS-{i+1:05d}" for i in range(len(dim_transport_mode))]
dim_transport_mode['TransportationModes'] = dim_transport_mode['DeliveryCompany']
dim_transport_mode['Routes'] = "Route Placeholder"

dim_transport_mode = dim_transport_mode[['TransportModeID', 'TransportationModes', 'Routes']]

dim_transport_mode.head()

In [ ]:
import numpy as np
import pandas as pd

# Define countries and cities mapping
countries = {
    "France": ["Paris", "Lyon", "Marseille", "Lille", "Nice", "Toulouse"],
    "US": ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix", "San Francisco"],
    "England": ["London", "Manchester", "Birmingham", "Leeds", "Liverpool", "Bristol"]
}

# Define postal code ranges per country
postal_code_ranges = {
    "France": (75000, 75999), 
    "US": (10000, 99999),     
    "England": (1000, 9999)   
}

# Function to generate postal codes based on the country
def generate_postal_code(country, city):
    if country in postal_code_ranges:
        # Get the range for the country
        start, end = postal_code_ranges[country]
        # Generate a postal code from this range
        return np.random.randint(start, end + 1)
    else:
        # Fallback for unknown countries
        return 99999

# Step 1: Extract unique clients and their store locations
unique_clients = transactions[['ClientID', 'ClientName', 'StoreCity', 'StoreCountry']].drop_duplicates()

# Step 2: Assign client locations based on store city and country
unique_clients['City'] = unique_clients['StoreCity']
unique_clients['Country'] = unique_clients['StoreCountry']

# Step 3: Generate logical gender proportions
np.random.seed(42)  # Set seed for reproducibility
gender_choices = np.random.choice(
    ['Female', 'Male'], 
    size=len(unique_clients), 
    p=[0.7, 0.3]  # 70% women, 30% men
)
unique_clients['Gender'] = gender_choices

# Step 4: Generate logical age distribution
# Flatten the ranges into a list of age options
age_options = (
    list(range(16, 41)) * int(0.6 * len(unique_clients)) +  # 60% young
    list(range(41, 61)) * int(0.3 * len(unique_clients)) +  # 30% middle-aged
    list(range(61, 91)) * int(0.1 * len(unique_clients))    # 10% elderly
)

# Shuffle the age options for randomness
np.random.shuffle(age_options)

# Assign ages to unique clients
unique_clients['Age'] = age_options[:len(unique_clients)]

# Step 5: Generate logical addresses
unique_clients['Address'] = unique_clients['City'] + " Main Street"

# Step 6: Ensure consistency for duplicate clients
# If a client appears multiple times, ensure their data is consistent
final_dim_client = unique_clients.groupby('ClientID').first().reset_index()

# Rename columns to match `dim_client` structure
final_dim_client = final_dim_client.rename(columns={
    'ClientName': 'FullName',
    'City': 'ClientCity',
    'Country': 'ClientCountry'
})

# Apply the postal code generation logic
final_dim_client['PostalCode'] = final_dim_client.apply(
    lambda row: generate_postal_code(row['ClientCountry'], row['ClientCity']),
    axis=1
)

# Select and arrange columns
final_dim_client = final_dim_client[['ClientID', 'FullName', 'Gender', 'Age', 'Address', 'ClientCity', 'PostalCode', 'ClientCountry']]

# Display the final dimension table
final_dim_client.head()


In [ ]:
# Create DIM_TEMPS
dim_temps = transactions[['TransactionDate']].drop_duplicates()

# Extract year, month, and day
dim_temps['TransactionDate'] = pd.to_datetime(dim_temps['TransactionDate'])
dim_temps['Jour'] = dim_temps['TransactionDate'].dt.day
dim_temps['Mois'] = dim_temps['TransactionDate'].dt.month
dim_temps['Annee'] = dim_temps['TransactionDate'].dt.year

# Display the result
dim_temps.head()

In [ ]:
# Create DIM_PAIEMENT
dim_paiement = transactions[['PaymentMethod']].drop_duplicates()

# Add unique PaiementID
dim_paiement['PaiementID'] = [f"PAY-{i+1:05d}" for i in range(len(dim_paiement))]

# Rearrange columns
dim_paiement = dim_paiement[['PaiementID', 'PaymentMethod']]

# Rename for consistency
dim_paiement = dim_paiement.rename(columns={'PaymentMethod': 'MethodePaiement'})

# Display the result
dim_paiement.head()

In [ ]:
# Create FACT_SUPPLY_CHAIN
fact_supply_chain = supply_chain[['SKU', 'StockLevel', 'DeliveryQuantity', 'TransportCost']]

# Add measures and derived fields
fact_supply_chain['RevenueGenerated'] = supply_chain['PurchasePrice']
fact_supply_chain['OrderQuantities'] = supply_chain['DeliveryQuantity']
fact_supply_chain['ShippingCosts'] = supply_chain['TransportCost']
fact_supply_chain['ManufacturingCosts'] = supply_chain['UnitPurchasePrice'] * supply_chain['DeliveryQuantity']
fact_supply_chain['DefectRates'] = np.random.uniform(0, 0.05, len(fact_supply_chain))  # Placeholder for defect rates

# Add unique FactID
fact_supply_chain['FactID'] = [f"FACT-SC-{i+1:05d}" for i in range(len(fact_supply_chain))]

# Rearrange columns
fact_supply_chain = fact_supply_chain[['FactID', 'SKU', 'StockLevel', 'OrderQuantities', 'ShippingCosts', 'RevenueGenerated', 'ManufacturingCosts', 'DefectRates']]

# Display the result
fact_supply_chain.head()

In [ ]:
# Create FACT_TRANSACTIONS
fact_transactions = transactions[['TransactionID', 'ClientID', 'TransactionDate', 'SKU', 'QuantityPurchased', 'TotalPurchasePrice']]

# Add derived measures
fact_transactions['PrixTotal'] = transactions['TotalSellingPrice']

# Add unique FactID
fact_transactions['FactID'] = [f"FACT-TR-{i+1:05d}" for i in range(len(fact_transactions))]

# Rename columns
fact_transactions = fact_transactions.rename(columns={'QuantityPurchased': 'Quantite', 'TotalPurchasePrice': 'PrixTotal'})

# Display the result
fact_transactions.head()

In [ ]:
# Display shapes of all tables
print("DIM_PRODUIT:", dim_produit.shape)
print("DIM_SUPPLIER:", dim_supplier.shape)
print("DIM_TRANSPORT_MODE:", dim_transport_mode.shape)
print("DIM_CLIENT:", dim_client.shape)
print("DIM_TEMPS:", dim_temps.shape)
print("DIM_PAIEMENT:", dim_paiement.shape)
print("FACT_SUPPLY_CHAIN:", fact_supply_chain.shape)
print("FACT_TRANSACTIONS:", fact_transactions.shape)

In [ ]:
# Create DIM_SHOPPING_MALL
dim_shopping_mall = transactions[['StoreName', 'StoreCity', 'StoreCountry']].drop_duplicates()
dim_shopping_mall['MallID'] = [f"MALL-{i+1:05d}" for i in range(len(dim_shopping_mall))]
dim_shopping_mall['Address'] = dim_shopping_mall['StoreCity'] + " Mall Street"

# Generate PostalCode
dim_shopping_mall['PostalCode'] = dim_shopping_mall.apply(
    lambda row: generate_postal_code(row['StoreCountry'], row['StoreCity']),
    axis=1
)

# Rearrange columns
dim_shopping_mall = dim_shopping_mall[['MallID', 'StoreName', 'Address', 'StoreCity', 'PostalCode', 'StoreCountry']]

# Rename columns for consistency
dim_shopping_mall = dim_shopping_mall.rename(columns={
    'StoreName': 'Name',
    'StoreCity': 'City',
    'StoreCountry': 'Country'
})

dim_shopping_mall.head()

In [ ]:
# Create DIM_TIME for delivery dates
dim_time_delivery = supply_chain[['DeliveryDate']].drop_duplicates()
dim_time_delivery['DeliveryDate'] = pd.to_datetime(dim_time_delivery['DeliveryDate'])
dim_time_delivery['Jour'] = dim_time_delivery['DeliveryDate'].dt.day
dim_time_delivery['Mois'] = dim_time_delivery['DeliveryDate'].dt.month
dim_time_delivery['Annee'] = dim_time_delivery['DeliveryDate'].dt.year

# Optionally, create a surrogate key
dim_time_delivery['DateID'] = dim_time_delivery['DeliveryDate'].dt.strftime('%Y%m%d').astype(int)

# Rearrange columns
dim_time_delivery = dim_time_delivery[['DateID', 'DeliveryDate', 'Jour', 'Mois', 'Annee']]

dim_time_delivery.head()

In [ ]:
print(dim_shopping_mall.info())
print(dim_shopping_mall.head())
print(dim_shopping_mall['PostalCode'].value_counts())